<a href="https://colab.research.google.com/github/BrunoReis136/API_fin_powered_by_OpenAI/blob/main/projeto_fin_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DADOS DA BIBLIOTECA YFINANCE

In [ ]:
import yfinance as yf
import pandas as pd

# --------------------------------------------
# Dicionário com nome das empresas e seus tickers
# --------------------------------------------
companies = {
    'Company': [
        'Apple Inc.',
        'Microsoft Corporation',
        'Alphabet Inc. (Google)',
        'Amazon.com Inc.',
        'Tesla Inc.',
        'Meta Platforms, Inc. (Facebook)',
        'NVIDIA Corporation',
        'Berkshire Hathaway Inc.',
        'Johnson & Johnson',
        'Walmart Inc.',
        'Procter & Gamble Co.',
        'Visa Inc.',
        'Pfizer Inc.',
        'Coca-Cola Company',
        'PepsiCo Inc.',
        'Intel Corporation',
        'Disney (The Walt Disney Company)',
        'Exxon Mobil Corporation',
        'Chevron Corporation',
        'AbbVie Inc.'
    ],
    'Code': [
        'AAPL',
        'MSFT',
        'GOOG',
        'AMZN',
        'TSLA',
        'META',
        'NVDA',
        'BRK-B',
        'JNJ',
        'WMT',
        'PG',
        'V',
        'PFE',
        'KO',
        'PEP',
        'INTC',
        'DIS',
        'XOM',
        'CVX',
        'ABBV'
    ]
}

# --------------------------------------------
# Estruturas para armazenar os dados coletados
# --------------------------------------------

# df_data irá armazenar um DataFrame por empresa
df_data = {}

# df_data_mean armazenará métricas agregadas por empresa
df_data_mean = {
    'Company': [],
    'Mean': [],   # média do preço médio (Mid)
    'Move': []    # média do movimento diário (Close - Open)
}

# --------------------------------------------
# Loop principal: coleta e processamento dos dados
# --------------------------------------------
for x in companies['Code']:
    # Cria o objeto da ação usando yfinance
    stock = yf.Ticker(x)

    # Baixa os últimos 5 dias de negociação
    data = stock.history(period="5d")

    # ----------------------------------------
    # Cálculos financeiros
    # ----------------------------------------

    # Preço médio do dia (Mid = (High + Low) / 2)
    mid = (data['High'] + data['Low']) / 2

    # Movimento diário do preço (Close - Open)
    move = data['Close'] - data['Open']

    # ----------------------------------------
    # Armazenamento dos dados individuais
    # ----------------------------------------

    # Cria uma cópia das colunas principais
    df_data[x] = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

    # Adiciona as métricas calculadas
    df_data[x]['Mid'] = mid
    df_data[x]['Move'] = move

    # ----------------------------------------
    # Armazena as médias para análise consolidada
    # ----------------------------------------

    # Nome da empresa correspondente ao ticker
    company_name = companies['Company'][companies['Code'].index(x)]

    df_data_mean['Company'].append(company_name)
    df_data_mean['Mean'].append(df_data[x]['Mid'].mean())
    df_data_mean['Move'].append(df_data[x]['Move'].mean())

# --------------------------------------------
# Consolidação dos dados em um único DataFrame
# --------------------------------------------

# Concatena todos os DataFrames das empresas em colunas
df = pd.concat(df_data, axis=1)

# Define o nome do índice como 'Date'
df.index.name = 'Date'

# Cria um MultiIndex nas colunas:
# Nível 0: Nome da empresa
# Nível 1: Tipo de dado (Open, High, Low, Close, Volume, Mid, Move)
df.columns = pd.MultiIndex.from_product(
    [companies['Company'], df.columns.levels[1]]
)

# Remove o timezone do índice de datas (compatível com Excel)
df.index = df.index.tz_localize(None)

# Arredonda os valores numéricos para 2 casas decimais
df = round(df, 2)

# --------------------------------------------
# Exportação dos dados
# --------------------------------------------

# Exporta o DataFrame consolidado para Excel
df.to_excel('StocksDays_MultiIndex.xlsx')

# --------------------------------------------
# Criação do DataFrame de médias por empresa
# --------------------------------------------

# Converte o dicionário de médias em DataFrame
dfm = pd.DataFrame(df_data_mean)

# Ordena as empresas pelo movimento médio (do maior para o menor)
dfm = dfm.sort_values(by='Move', ascending=False)

# Reseta o índice para melhor visualização
dfm.reset_index(drop=True, inplace=True)

# --------------------------------------------
# Exibição dos resultados
# --------------------------------------------

print(df_data_mean)

print(df)

print('\n', 100 * '-', '\n')

print(dfm)


# USANDO API DA FINNHUB

In [ ]:
API_KEY = "SEU_API_KEY"
url = f"https://finnhub.io/api/v1/calendar/earnings?symbol=AAPL&token={API_KEY}"
resp = requests.get(url).json()
print(resp)